In [51]:
import sys
from bs4 import BeautifulSoup
import urllib.request
import requests
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fname = fm.FontProperties(fname='malgun.ttf').get_name()
import sqlite3
from IPython.display import Image
mpl.rc('font',family=fname)
import matplotlib.gridspec as gridspec
from jupyterthemes import jtplot
jtplot.style()
%matplotlib inline
import platform
platform.system()
# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'
import re

### 매도/매수/순매수 (한국은행) 파일들 행열변환 및 누적순매수 구하기

In [3]:
maedo = pd.read_csv("maedo.csv",encoding='CP949')
maedo

,통계표,항목명1,항목명2,단위,변환,2018/09,2018/10,2018/11,2018/12,2019/01,...,2019/11,2019/12,2020/01,2020/02,2020/03,2020/04,2020/05,2020/06,2020/07,2020/08
0,6.1.3 투자자별 주식거래,기관투자자(매도),거래량,천주,원자료,"463,930","637,169","631,475","567,593","767,544",...,"610,691","608,640","795,743","863,490","1,328,075","820,561","730,705","1,127,406","899,292","1,020,601"
1,6.1.3 투자자별 주식거래,기관투자자(매도),거래대금,백만원,원자료,"21,738,513","28,440,410","26,752,516","23,321,422","29,659,073",...,"22,466,738","22,655,876","34,055,564","36,151,350","47,750,913","32,342,499","31,906,416","50,631,634","44,062,849","52,594,689"
2,6.1.3 투자자별 주식거래,개인(매도),거래량,천주,원자료,"4,124,203","5,221,373","5,180,401","6,065,660","7,564,319",...,"8,831,863","8,361,963","10,827,318","11,355,097","12,863,057","18,039,382","13,726,551","16,759,031","16,283,916","14,865,701"
3,6.1.3 투자자별 주식거래,개인(매도),거래대금,백만원,원자료,"49,722,354","55,931,757","53,664,002","46,923,235","61,777,978",...,"57,035,793","48,267,496","61,479,601","71,591,345","112,306,528","145,980,478","120,231,528","193,240,887","215,207,435","225,608,813"
4,6.1.3 투자자별 주식거래,외국인(매도),거래량,천주,원자료,"792,249","1,104,143","1,116,264","923,756","1,021,412",...,"1,213,399","1,132,933","1,364,161","1,613,681","2,789,747","2,023,830","1,616,039","1,815,743","1,474,532","1,609,091"
5,6.1.3 투자자별 주식거래,외국인(매도),거래대금,백만원,원자료,"30,020,323","40,719,057","35,096,871","28,525,485","30,812,607",...,"31,447,403","29,565,931","32,189,303","41,118,506","60,481,882","35,951,855","35,309,053","41,120,950","37,788,501","42,770,238"
6,6.1.3 투자자별 주식거래,기타외국인(매도),거래량,천주,원자료,"7,764","10,034","9,109","11,708","15,415",...,"17,824","20,066","23,249","24,308","30,239","44,962","36,411","45,497","42,227","34,739"
7,6.1.3 투자자별 주식거래,기타외국인(매도),거래대금,백만원,원자료,"120,386","113,849","107,032","108,432","148,252",...,"128,025","127,409","153,788","182,930","262,679","348,162","324,497","463,668","558,276","528,555"
8,6.1.3 투자자별 주식거래,기타법인(매도),거래량,천주,원자료,"45,080","49,377","41,709","63,162","86,508",...,"63,870","65,151","85,595","103,191","79,908","116,386","96,790","116,027","166,819","112,992"
9,6.1.3 투자자별 주식거래,기타법인(매도),거래대금,백만원,원자료,"1,578,384","2,928,148","819,023","863,622","871,640",...,"581,096","693,238","815,204","2,613,585","1,325,868","1,436,116","1,417,860","2,184,839","2,152,021","2,446,592"


In [13]:
new_maedo = maedo.T
new_maedo.head(7)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
통계표,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,...,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래
항목명1,기관투자자(매도),기관투자자(매도),개인(매도),개인(매도),외국인(매도),외국인(매도),기타외국인(매도),기타외국인(매도),기타법인(매도),기타법인(매도),...,기관투자자(순매수),기관투자자(순매수),개인(순매수),개인(순매수),외국인(순매수),외국인(순매수),기타외국인(순매수),기타외국인(순매수),기타법인(순매수),기타법인(순매수)
항목명2,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,...,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금
단위,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원,...,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원
변환,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,...,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료
2018/09,"463,930","21,738,513","4,124,203","49,722,354","792,249","30,020,323","7,764","120,386","45,080","1,578,384",...,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
2018/10,"637,169","28,440,410","5,221,373","55,931,757","1,104,143","40,719,057","10,034","113,849","49,377","2,928,148",...,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"


In [20]:
re_maedo = new_maedo.reset_index()
re_maedo.head(7)

,index,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,통계표,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,...,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래
1,항목명1,기관투자자(매도),기관투자자(매도),개인(매도),개인(매도),외국인(매도),외국인(매도),기타외국인(매도),기타외국인(매도),기타법인(매도),...,기관투자자(순매수),기관투자자(순매수),개인(순매수),개인(순매수),외국인(순매수),외국인(순매수),기타외국인(순매수),기타외국인(순매수),기타법인(순매수),기타법인(순매수)
2,항목명2,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,...,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금
3,단위,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,...,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원
4,변환,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,...,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료
5,2018/09,"463,930","21,738,513","4,124,203","49,722,354","792,249","30,020,323","7,764","120,386","45,080",...,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
6,2018/10,"637,169","28,440,410","5,221,373","55,931,757","1,104,143","40,719,057","10,034","113,849","49,377",...,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"


In [23]:
maedo_ = re_maedo.drop([re_maedo.index[0],re_maedo.index[1],re_maedo.index[2],re_maedo.index[3],re_maedo.index[4]])
maedo_.head(7)

,index,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
5,2018/09,"463,930","21,738,513","4,124,203","49,722,354","792,249","30,020,323","7,764","120,386","45,080",...,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
6,2018/10,"637,169","28,440,410","5,221,373","55,931,757","1,104,143","40,719,057","10,034","113,849","49,377",...,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
7,2018/11,"631,475","26,752,516","5,180,401","53,664,002","1,116,264","35,096,871","9,109","107,032","41,709",...,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
8,2018/12,"567,593","23,321,422","6,065,660","46,923,235","923,756","28,525,485","11,708","108,432","63,162",...,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
9,2019/01,"767,544","29,659,073","7,564,319","61,777,978","1,021,412","30,812,607","15,415","148,252","86,508",...,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"
10,2019/02,"536,465","21,943,755","5,417,014","45,699,594","857,260","27,118,281","12,318","118,299","59,498",...,"-17,872","541,546","55,291","-826,496","-13,077","140,834",-693,"-6,077","-23,649","150,193"
11,2019/03,"635,124","25,774,280","4,810,184","40,946,832","930,013","29,228,049","9,091","90,111","73,644",...,"-26,795","-994,507","31,582","590,215","10,674","300,926",168,-397,"-15,628","103,763"


In [25]:
# 칼럼명 변경

maedo_.columns = ['해당월','기관투자자(매도/거래량)','기관투자자(매도/거래대금)',
                  '개인(매도/거래량)','개인(매도/거래대금)',
                  '외국인(매도/거래량)','외국인(매도/거래대금)',
                  '기타외국인(매도/거래량)','기타외국인(매도/거래대금)',
                  '기타법인(매도/거래량)','기타법인(매도/거래대금)',
                  '기관투자자(매수/거래량)','기관투자자(매수/거래대금)',
                  '개인(매수/거래량)','개인(매수/거래대금)',
                  '외국인(매수/거래량)','외국인(매수/거래대금)',
                  '기타외국인(매수/거래량)','기타외국인(매수/거래대금)',
                  '기타법인(매수/거래량)','기타법인(매수/거래대금)',
                  '기관투자자(순매수/거래량)','기관투자자(순매수/거래대금)',
                  '개인(순매수/거래량)','개인(순매수/거래대금)',
                  '외국인(순매수/거래량)','외국인(순매수/거래대금)',
                  '기타외국인(순매수/거래량)','기타외국인(순매수/거래대금)',
                  '기타법인(순매수/거래량)','기타법인(순매수/거래대금)']

In [28]:
maedo_.head(5)

,해당월,기관투자자(매도/거래량),기관투자자(매도/거래대금),개인(매도/거래량),개인(매도/거래대금),외국인(매도/거래량),외국인(매도/거래대금),기타외국인(매도/거래량),기타외국인(매도/거래대금),기타법인(매도/거래량),...,기관투자자(순매수/거래량),기관투자자(순매수/거래대금),개인(순매수/거래량),개인(순매수/거래대금),외국인(순매수/거래량),외국인(순매수/거래대금),기타외국인(순매수/거래량),기타외국인(순매수/거래대금),기타법인(순매수/거래량),기타법인(순매수/거래대금)
5,2018/09,"463,930","21,738,513","4,124,203","49,722,354","792,249","30,020,323","7,764","120,386","45,080",...,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
6,2018/10,"637,169","28,440,410","5,221,373","55,931,757","1,104,143","40,719,057","10,034","113,849","49,377",...,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
7,2018/11,"631,475","26,752,516","5,180,401","53,664,002","1,116,264","35,096,871","9,109","107,032","41,709",...,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
8,2018/12,"567,593","23,321,422","6,065,660","46,923,235","923,756","28,525,485","11,708","108,432","63,162",...,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
9,2019/01,"767,544","29,659,073","7,564,319","61,777,978","1,021,412","30,812,607","15,415","148,252","86,508",...,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"


In [27]:
maedo_fin = maedo_
maedo_fin

,해당월,기관투자자(매도/거래량),기관투자자(매도/거래대금),개인(매도/거래량),개인(매도/거래대금),외국인(매도/거래량),외국인(매도/거래대금),기타외국인(매도/거래량),기타외국인(매도/거래대금),기타법인(매도/거래량),...,기관투자자(순매수/거래량),기관투자자(순매수/거래대금),개인(순매수/거래량),개인(순매수/거래대금),외국인(순매수/거래량),외국인(순매수/거래대금),기타외국인(순매수/거래량),기타외국인(순매수/거래대금),기타법인(순매수/거래량),기타법인(순매수/거래대금)
5,2018/09,"463,930","21,738,513","4,124,203","49,722,354","792,249","30,020,323","7,764","120,386","45,080",...,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
6,2018/10,"637,169","28,440,410","5,221,373","55,931,757","1,104,143","40,719,057","10,034","113,849","49,377",...,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
7,2018/11,"631,475","26,752,516","5,180,401","53,664,002","1,116,264","35,096,871","9,109","107,032","41,709",...,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
8,2018/12,"567,593","23,321,422","6,065,660","46,923,235","923,756","28,525,485","11,708","108,432","63,162",...,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
9,2019/01,"767,544","29,659,073","7,564,319","61,777,978","1,021,412","30,812,607","15,415","148,252","86,508",...,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"
10,2019/02,"536,465","21,943,755","5,417,014","45,699,594","857,260","27,118,281","12,318","118,299","59,498",...,"-17,872","541,546","55,291","-826,496","-13,077","140,834",-693,"-6,077","-23,649","150,193"
11,2019/03,"635,124","25,774,280","4,810,184","40,946,832","930,013","29,228,049","9,091","90,111","73,644",...,"-26,795","-994,507","31,582","590,215","10,674","300,926",168,-397,"-15,628","103,763"
12,2019/04,"697,869","27,004,481","7,916,701","61,205,745","911,913","29,104,189","17,282","136,022","145,983",...,"-20,559","-654,037","49,547","-1,747,825","58,509","2,392,135",-90,"-10,017","-87,407","19,744"
13,2019/05,"704,402","25,691,163","9,176,499","50,121,982","1,268,337","36,521,584","18,538","99,900","75,116",...,"-45,430","-462,639","129,047","2,799,941","-69,187","-2,480,729","1,192","13,967","-15,623","129,460"
14,2019/06,"551,935","19,970,200","9,618,449","41,974,243","999,229","24,613,715","23,850","139,470","63,224",...,"15,348","1,511,951","-15,366","-1,970,077","22,015","758,298","-1,454","-50,071","-20,543","-250,101"


In [55]:
maedo_fin_fin = maedo_fin.reset_index()
maedo_fin_fin.drop(['index'],axis='columns',inplace=True)

In [56]:
maedo_fin_fin

,해당월,기관투자자(매도/거래량),기관투자자(매도/거래대금),개인(매도/거래량),개인(매도/거래대금),외국인(매도/거래량),외국인(매도/거래대금),기타외국인(매도/거래량),기타외국인(매도/거래대금),기타법인(매도/거래량),...,기관투자자(순매수/거래량),기관투자자(순매수/거래대금),개인(순매수/거래량),개인(순매수/거래대금),외국인(순매수/거래량),외국인(순매수/거래대금),기타외국인(순매수/거래량),기타외국인(순매수/거래대금),기타법인(순매수/거래량),기타법인(순매수/거래대금)
0,2018/09,"463,930","21,738,513","4,124,203","49,722,354","792,249","30,020,323","7,764","120,386","45,080",...,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
1,2018/10,"637,169","28,440,410","5,221,373","55,931,757","1,104,143","40,719,057","10,034","113,849","49,377",...,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
2,2018/11,"631,475","26,752,516","5,180,401","53,664,002","1,116,264","35,096,871","9,109","107,032","41,709",...,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
3,2018/12,"567,593","23,321,422","6,065,660","46,923,235","923,756","28,525,485","11,708","108,432","63,162",...,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
4,2019/01,"767,544","29,659,073","7,564,319","61,777,978","1,021,412","30,812,607","15,415","148,252","86,508",...,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"
5,2019/02,"536,465","21,943,755","5,417,014","45,699,594","857,260","27,118,281","12,318","118,299","59,498",...,"-17,872","541,546","55,291","-826,496","-13,077","140,834",-693,"-6,077","-23,649","150,193"
6,2019/03,"635,124","25,774,280","4,810,184","40,946,832","930,013","29,228,049","9,091","90,111","73,644",...,"-26,795","-994,507","31,582","590,215","10,674","300,926",168,-397,"-15,628","103,763"
7,2019/04,"697,869","27,004,481","7,916,701","61,205,745","911,913","29,104,189","17,282","136,022","145,983",...,"-20,559","-654,037","49,547","-1,747,825","58,509","2,392,135",-90,"-10,017","-87,407","19,744"
8,2019/05,"704,402","25,691,163","9,176,499","50,121,982","1,268,337","36,521,584","18,538","99,900","75,116",...,"-45,430","-462,639","129,047","2,799,941","-69,187","-2,480,729","1,192","13,967","-15,623","129,460"
9,2019/06,"551,935","19,970,200","9,618,449","41,974,243","999,229","24,613,715","23,850","139,470","63,224",...,"15,348","1,511,951","-15,366","-1,970,077","22,015","758,298","-1,454","-50,071","-20,543","-250,101"


In [59]:
maedo_fin_fin.to_csv('C:/Users/starfish/Desktop/월별_투자자별_매도매수순매수.csv',index=False,encoding='utf-8-sig')

----------------------------------------

In [32]:
maesoo = pd.read_csv("maesoo.csv",encoding='CP949')
maesoo.head(7)

,통계표,항목명1,항목명2,단위,변환,2018/09,2018/10,2018/11,2018/12,2019/01,...,2019/11,2019/12,2020/01,2020/02,2020/03,2020/04,2020/05,2020/06,2020/07,2020/08
0,6.1.3 투자자별 주식거래,기관투자자(매수),거래량,천주,원자료,"467,208","644,065","621,208","598,565","709,965",...,"609,000","652,698","656,563","764,739","1,230,665","812,367","737,099","1,032,054","809,295","886,116"
1,6.1.3 투자자별 주식거래,기관투자자(매수),거래대금,백만원,원자료,"22,373,600","30,230,135","26,534,632","24,496,609","28,815,928",...,"24,776,429","25,418,585","28,981,383","33,915,260","47,867,558","32,328,809","31,718,274","47,920,882","40,999,109","49,030,722"
2,6.1.3 투자자별 주식거래,개인(매수),거래량,천주,원자료,"4,117,139","5,210,249","5,174,406","6,042,091","7,539,567",...,"8,951,416","8,311,271","11,071,279","11,605,103","13,119,178","18,202,245","13,892,302","16,969,746","16,411,987","15,047,495"
3,6.1.3 투자자별 주식거래,개인(매수),거래대금,백만원,원자료,"49,689,208","57,841,723","53,292,315","45,689,344","58,548,458",...,"57,174,412","44,439,960","65,962,645","76,488,701","123,493,458","149,792,898","124,015,009","197,055,237","217,446,353","231,779,569"
4,6.1.3 투자자별 주식거래,외국인(매수),거래량,천주,원자료,"803,411","1,091,707","1,121,874","924,230","1,141,976",...,"1,108,053","1,142,365","1,286,949","1,481,214","2,535,172","1,879,899","1,449,379","1,704,317","1,520,195","1,574,401"
5,6.1.3 투자자별 주식거래,외국인(매수),거래대금,백만원,원자료,"29,722,838","36,720,280","35,457,036","28,474,948","34,862,640",...,"28,276,606","30,184,318","32,494,024","37,805,348","47,926,872","31,851,776","31,425,259","39,902,125","38,867,596","39,923,350"
6,6.1.3 투자자별 주식거래,기타외국인(매수),거래량,천주,원자료,"7,822","9,922","9,080","11,585","14,916",...,"18,484","19,681","22,911","24,413","32,495","43,613","38,927","47,079","40,650","34,620"


In [33]:
new_maesoo = maesoo.T
new_maesoo.head(7)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
통계표,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래
항목명1,기관투자자(매수),기관투자자(매수),개인(매수),개인(매수),외국인(매수),외국인(매수),기타외국인(매수),기타외국인(매수),기타법인(매수),기타법인(매수),기관투자자(순매수),기관투자자(순매수),개인(순매수),개인(순매수),외국인(순매수),외국인(순매수),기타외국인(순매수),기타외국인(순매수),기타법인(순매수),기타법인(순매수)
항목명2,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금
단위,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원
변환,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료
2018/09,"467,208","22,373,600","4,117,139","49,689,208","803,411","29,722,838","7,822","110,382","37,646","1,283,932","3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
2018/10,"644,065","30,230,135","5,210,249","57,841,723","1,091,707","36,720,280","9,922","124,000","66,153","3,217,083","6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"


In [34]:
re_maesoo = new_maesoo.reset_index()
re_maesoo.head(7)

,index,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,통계표,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,...,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래
1,항목명1,기관투자자(매수),기관투자자(매수),개인(매수),개인(매수),외국인(매수),외국인(매수),기타외국인(매수),기타외국인(매수),기타법인(매수),...,기관투자자(순매수),기관투자자(순매수),개인(순매수),개인(순매수),외국인(순매수),외국인(순매수),기타외국인(순매수),기타외국인(순매수),기타법인(순매수),기타법인(순매수)
2,항목명2,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,...,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금
3,단위,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,...,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원
4,변환,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,...,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료
5,2018/09,"467,208","22,373,600","4,117,139","49,689,208","803,411","29,722,838","7,822","110,382","37,646",...,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
6,2018/10,"644,065","30,230,135","5,210,249","57,841,723","1,091,707","36,720,280","9,922","124,000","66,153",...,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"


In [35]:
maesoo_ = re_maesoo.drop([re_maesoo.index[0],re_maesoo.index[1],re_maesoo.index[2],re_maesoo.index[3],re_maesoo.index[4]])
maesoo_.head(7)

,index,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
5,2018/09,"467,208","22,373,600","4,117,139","49,689,208","803,411","29,722,838","7,822","110,382","37,646",...,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
6,2018/10,"644,065","30,230,135","5,210,249","57,841,723","1,091,707","36,720,280","9,922","124,000","66,153",...,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
7,2018/11,"621,208","26,534,632","5,174,406","53,292,315","1,121,874","35,457,036","9,080","100,260","52,392",...,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
8,2018/12,"598,565","24,496,609","6,042,091","45,689,344","924,230","28,474,948","11,585","99,864","55,408",...,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
9,2019/01,"709,965","28,815,928","7,539,567","58,548,458","1,141,976","34,862,640","14,916","131,777","48,773",...,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"
10,2019/02,"518,593","22,485,301","5,472,305","44,873,098","844,183","27,259,116","11,624","112,222","35,849",...,"-17,872","541,546","55,291","-826,496","-13,077","140,834",-693,"-6,077","-23,649","150,193"
11,2019/03,"608,329","24,779,774","4,841,766","41,537,047","940,687","29,528,974","9,258","89,713","58,016",...,"-26,795","-994,507","31,582","590,215","10,674","300,926",168,-397,"-15,628","103,763"


In [36]:
maesoo_.columns = ['해당월',
                  '기관투자자(매수/거래량)','기관투자자(매수/거래대금)',
                  '개인(매수/거래량)','개인(매수/거래대금)',
                  '외국인(매수/거래량)','외국인(매수/거래대금)',
                  '기타외국인(매수/거래량)','기타외국인(매수/거래대금)',
                  '기타법인(매수/거래량)','기타법인(매수/거래대금)',
                  '기관투자자(순매수/거래량)','기관투자자(순매수/거래대금)',
                  '개인(순매수/거래량)','개인(순매수/거래대금)',
                  '외국인(순매수/거래량)','외국인(순매수/거래대금)',
                  '기타외국인(순매수/거래량)','기타외국인(순매수/거래대금)',
                  '기타법인(순매수/거래량)','기타법인(순매수/거래대금)']

In [37]:
maesoo_.head(5)

,해당월,기관투자자(매수/거래량),기관투자자(매수/거래대금),개인(매수/거래량),개인(매수/거래대금),외국인(매수/거래량),외국인(매수/거래대금),기타외국인(매수/거래량),기타외국인(매수/거래대금),기타법인(매수/거래량),...,기관투자자(순매수/거래량),기관투자자(순매수/거래대금),개인(순매수/거래량),개인(순매수/거래대금),외국인(순매수/거래량),외국인(순매수/거래대금),기타외국인(순매수/거래량),기타외국인(순매수/거래대금),기타법인(순매수/거래량),기타법인(순매수/거래대금)
5,2018/09,"467,208","22,373,600","4,117,139","49,689,208","803,411","29,722,838","7,822","110,382","37,646",...,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
6,2018/10,"644,065","30,230,135","5,210,249","57,841,723","1,091,707","36,720,280","9,922","124,000","66,153",...,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
7,2018/11,"621,208","26,534,632","5,174,406","53,292,315","1,121,874","35,457,036","9,080","100,260","52,392",...,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
8,2018/12,"598,565","24,496,609","6,042,091","45,689,344","924,230","28,474,948","11,585","99,864","55,408",...,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
9,2019/01,"709,965","28,815,928","7,539,567","58,548,458","1,141,976","34,862,640","14,916","131,777","48,773",...,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"


In [40]:
maesoo_fin = maesoo_
maesoo_fin

,해당월,기관투자자(매수/거래량),기관투자자(매수/거래대금),개인(매수/거래량),개인(매수/거래대금),외국인(매수/거래량),외국인(매수/거래대금),기타외국인(매수/거래량),기타외국인(매수/거래대금),기타법인(매수/거래량),...,기관투자자(순매수/거래량),기관투자자(순매수/거래대금),개인(순매수/거래량),개인(순매수/거래대금),외국인(순매수/거래량),외국인(순매수/거래대금),기타외국인(순매수/거래량),기타외국인(순매수/거래대금),기타법인(순매수/거래량),기타법인(순매수/거래대금)
5,2018/09,"467,208","22,373,600","4,117,139","49,689,208","803,411","29,722,838","7,822","110,382","37,646",...,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
6,2018/10,"644,065","30,230,135","5,210,249","57,841,723","1,091,707","36,720,280","9,922","124,000","66,153",...,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
7,2018/11,"621,208","26,534,632","5,174,406","53,292,315","1,121,874","35,457,036","9,080","100,260","52,392",...,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
8,2018/12,"598,565","24,496,609","6,042,091","45,689,344","924,230","28,474,948","11,585","99,864","55,408",...,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
9,2019/01,"709,965","28,815,928","7,539,567","58,548,458","1,141,976","34,862,640","14,916","131,777","48,773",...,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"
10,2019/02,"518,593","22,485,301","5,472,305","44,873,098","844,183","27,259,116","11,624","112,222","35,849",...,"-17,872","541,546","55,291","-826,496","-13,077","140,834",-693,"-6,077","-23,649","150,193"
11,2019/03,"608,329","24,779,774","4,841,766","41,537,047","940,687","29,528,974","9,258","89,713","58,016",...,"-26,795","-994,507","31,582","590,215","10,674","300,926",168,-397,"-15,628","103,763"
12,2019/04,"677,309","26,350,444","7,966,248","59,457,920","970,422","31,496,323","17,192","126,005","58,577",...,"-20,559","-654,037","49,547","-1,747,825","58,509","2,392,135",-90,"-10,017","-87,407","19,744"
13,2019/05,"658,973","25,228,524","9,305,547","52,921,922","1,199,150","34,040,855","19,730","113,867","59,493",...,"-45,430","-462,639","129,047","2,799,941","-69,187","-2,480,729","1,192","13,967","-15,623","129,460"
14,2019/06,"567,284","21,482,151","9,603,083","40,004,166","1,021,244","25,372,014","22,396","89,398","42,681",...,"15,348","1,511,951","-15,366","-1,970,077","22,015","758,298","-1,454","-50,071","-20,543","-250,101"


In [60]:
maesoo_fin_fin = maesoo_fin.reset_index()
maesoo_fin_fin.drop(['index'],axis='columns',inplace=True)

In [61]:
maesoo_fin_fin.to_csv('.../월별_투자자별_매수순매수.csv',index=False,encoding='utf-8-sig')

--------------------

### 분석의 용이를 위해 숫자 데이터의 타입을 변경할 필요성 有

In [65]:
df1 = maedo_fin_fin
df1

,해당월,기관투자자(매도/거래량),기관투자자(매도/거래대금),개인(매도/거래량),개인(매도/거래대금),외국인(매도/거래량),외국인(매도/거래대금),기타외국인(매도/거래량),기타외국인(매도/거래대금),기타법인(매도/거래량),...,기관투자자(순매수/거래량),기관투자자(순매수/거래대금),개인(순매수/거래량),개인(순매수/거래대금),외국인(순매수/거래량),외국인(순매수/거래대금),기타외국인(순매수/거래량),기타외국인(순매수/거래대금),기타법인(순매수/거래량),기타법인(순매수/거래대금)
0,2018/09,"463,930","21,738,513","4,124,203","49,722,354","792,249","30,020,323","7,764","120,386","45,080",...,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
1,2018/10,"637,169","28,440,410","5,221,373","55,931,757","1,104,143","40,719,057","10,034","113,849","49,377",...,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
2,2018/11,"631,475","26,752,516","5,180,401","53,664,002","1,116,264","35,096,871","9,109","107,032","41,709",...,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
3,2018/12,"567,593","23,321,422","6,065,660","46,923,235","923,756","28,525,485","11,708","108,432","63,162",...,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
4,2019/01,"767,544","29,659,073","7,564,319","61,777,978","1,021,412","30,812,607","15,415","148,252","86,508",...,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"
5,2019/02,"536,465","21,943,755","5,417,014","45,699,594","857,260","27,118,281","12,318","118,299","59,498",...,"-17,872","541,546","55,291","-826,496","-13,077","140,834",-693,"-6,077","-23,649","150,193"
6,2019/03,"635,124","25,774,280","4,810,184","40,946,832","930,013","29,228,049","9,091","90,111","73,644",...,"-26,795","-994,507","31,582","590,215","10,674","300,926",168,-397,"-15,628","103,763"
7,2019/04,"697,869","27,004,481","7,916,701","61,205,745","911,913","29,104,189","17,282","136,022","145,983",...,"-20,559","-654,037","49,547","-1,747,825","58,509","2,392,135",-90,"-10,017","-87,407","19,744"
8,2019/05,"704,402","25,691,163","9,176,499","50,121,982","1,268,337","36,521,584","18,538","99,900","75,116",...,"-45,430","-462,639","129,047","2,799,941","-69,187","-2,480,729","1,192","13,967","-15,623","129,460"
9,2019/06,"551,935","19,970,200","9,618,449","41,974,243","999,229","24,613,715","23,850","139,470","63,224",...,"15,348","1,511,951","-15,366","-1,970,077","22,015","758,298","-1,454","-50,071","-20,543","-250,101"


In [66]:
new_g1 = df1['기관투자자(매도/거래량)'].str.replace(',', '').astype('int64')
new_g2 = df1['기관투자자(매도/거래대금)'].str.replace(',', '').astype('int64')
new_g3 = df1['개인(매도/거래량)'].str.replace(',', '').astype('int64')
new_g4 = df1['개인(매도/거래대금)'].str.replace(',', '').astype('int64')
new_g5 = df1['외국인(매도/거래량)'].str.replace(',', '').astype('int64')
new_g6 = df1['외국인(매도/거래대금)'].str.replace(',', '').astype('int64')
new_g7 = df1['기타외국인(매도/거래량)'].str.replace(',', '').astype('int64')
new_g8 = df1['기타외국인(매도/거래대금)'].str.replace(',', '').astype('int64')
new_g9 = df1['기타법인(매도/거래량)'].str.replace(',', '').astype('int64')
new_g10 = df1['기타법인(매도/거래대금)'].str.replace(',', '').astype('int64')
new_g11 = df1['기관투자자(매수/거래량)'].str.replace(',', '').astype('int64')
new_g12 = df1['기관투자자(매수/거래대금)'].str.replace(',', '').astype('int64')
new_g13 = df1['개인(매수/거래량)'].str.replace(',', '').astype('int64')
new_g14 = df1['개인(매수/거래대금)'].str.replace(',', '').astype('int64')
new_g15 = df1['외국인(매수/거래량)'].str.replace(',', '').astype('int64')
new_g16 = df1['외국인(매수/거래대금)'].str.replace(',', '').astype('int64')
new_g17 = df1['기타외국인(매수/거래량)'].str.replace(',', '').astype('int64')
new_g18 = df1['기타외국인(매수/거래대금)'].str.replace(',', '').astype('int64')
new_g19 = df1['기타법인(매수/거래량)'].str.replace(',', '').astype('int64')
new_g20 = df1['기타법인(매수/거래대금)'].str.replace(',', '').astype('int64')
new_g21 = df1['기관투자자(순매수/거래량)'].str.replace(',', '').astype('int64')
new_g22 = df1['기관투자자(순매수/거래대금)'].str.replace(',', '').astype('int64')
new_g23 = df1['개인(순매수/거래량)'].str.replace(',', '').astype('int64')
new_g24 = df1['개인(순매수/거래대금)'].str.replace(',', '').astype('int64')
new_g25 = df1['외국인(순매수/거래량)'].str.replace(',', '').astype('int64')
new_g26 = df1['외국인(순매수/거래대금)'].str.replace(',', '').astype('int64')
new_g27 = df1['기타외국인(순매수/거래량)'].str.replace(',', '').astype('int64')
new_g28 = df1['기타외국인(순매수/거래대금)'].str.replace(',', '').astype('int64')
new_g29 = df1['기타법인(순매수/거래량)'].str.replace(',', '').astype('int64')
new_g30 = df1['기타법인(순매수/거래대금)'].str.replace(',', '').astype('int64')

In [67]:
df1['new_기관투자자(매도/거래량)']=new_g1
df1['new_기관투자자(매도/거래대금)']=new_g2
df1['new_개인(매도/거래량)']=new_g3
df1['new_개인(매도/거래대금)']=new_g4
df1['new_외국인(매도/거래량)']=new_g5
df1['new_외국인(매도/거래대금)']=new_g6
df1['new_기타외국인(매도/거래량)']=new_g7
df1['new_기타외국인(매도/거래대금)']=new_g8
df1['new_기타법인(매도/거래량)']=new_g9
df1['new_기타법인(매도/거래대금)']=new_g10
df1['new_기관투자자(매수/거래량)']=new_g11
df1['new_기관투자자(매수/거래대금)']=new_g12
df1['new_개인(매수/거래량)']=new_g13
df1['new_개인(매수/거래대금)']=new_g14
df1['new_외국인(매수/거래량)']=new_g15
df1['new_외국인(매수/거래대금)']=new_g16
df1['new_기타외국인(매수/거래량)']=new_g17
df1['new_기타외국인(매수/거래대금)']=new_g18
df1['new_기타법인(매수/거래량)']=new_g19
df1['new_기타법인(매수/거래대금)']=new_g20
df1['new_기관투자자(순매수/거래량)']=new_g21
df1['new_기관투자자(순매수/거래대금)']=new_g22
df1['new_개인(순매수/거래량)']=new_g23
df1['new_개인(순매수/거래대금)']=new_g24
df1['new_외국인(순매수/거래량)']=new_g25
df1['new_외국인(순매수/거래대금)']=new_g26
df1['new_기타외국인(순매수/거래량)']=new_g27
df1['new_기타외국인(순매수/거래대금)']=new_g28
df1['new_기타법인(순매수/거래량)']=new_g29
df1['new_기타법인(순매수/거래대금)']=new_g30

In [68]:
df1.head(5)

,해당월,기관투자자(매도/거래량),기관투자자(매도/거래대금),개인(매도/거래량),개인(매도/거래대금),외국인(매도/거래량),외국인(매도/거래대금),기타외국인(매도/거래량),기타외국인(매도/거래대금),기타법인(매도/거래량),...,new_기관투자자(순매수/거래량),new_기관투자자(순매수/거래대금),new_개인(순매수/거래량),new_개인(순매수/거래대금),new_외국인(순매수/거래량),new_외국인(순매수/거래대금),new_기타외국인(순매수/거래량),new_기타외국인(순매수/거래대금),new_기타법인(순매수/거래량),new_기타법인(순매수/거래대금)
0,2018/09,"463,930","21,738,513","4,124,203","49,722,354","792,249","30,020,323","7,764","120,386","45,080",...,3279,635087,-7064,-33147,11161,-297485,58,-10004,-7434,-294452
1,2018/10,"637,169","28,440,410","5,221,373","55,931,757","1,104,143","40,719,057","10,034","113,849","49,377",...,6896,1789725,-11124,1909966,-12437,-3998777,-112,10151,16777,288935
2,2018/11,"631,475","26,752,516","5,180,401","53,664,002","1,116,264","35,096,871","9,109","107,032","41,709",...,-10267,-217884,-5996,-371686,5610,360165,-30,-6772,10683,236178
3,2018/12,"567,593","23,321,422","6,065,660","46,923,235","923,756","28,525,485","11,708","108,432","63,162",...,30972,1175187,-23569,-1233891,473,-50537,-122,-8567,-7754,117808
4,2019/01,"767,544","29,659,073","7,564,319","61,777,978","1,021,412","30,812,607","15,415","148,252","86,508",...,-57579,-843145,-24751,-3229521,120564,4050033,-499,-16475,-37734,39108


In [69]:
df_re1 = df1[['해당월','new_기관투자자(매도/거래량)','new_기관투자자(매도/거래대금)',
              'new_개인(매도/거래량)','new_개인(매도/거래대금)','new_외국인(매도/거래량)',
            'new_외국인(매도/거래대금)','new_기타외국인(매도/거래량)','new_기타외국인(매도/거래대금)',
            'new_기타법인(매도/거래량)','new_기타법인(매도/거래대금)','new_기관투자자(매수/거래량)','new_기관투자자(매수/거래대금)',
             'new_개인(매수/거래량)','new_개인(매수/거래대금)','new_외국인(매수/거래량)',
            'new_외국인(매수/거래대금)','new_기타외국인(매수/거래량)','new_기타외국인(매수/거래대금)',
            'new_기타법인(매수/거래량)','new_기타법인(매수/거래대금)','new_기관투자자(순매수/거래량)','new_기관투자자(순매수/거래대금)',
             'new_개인(순매수/거래량)','new_개인(순매수/거래대금)','new_외국인(순매수/거래량)',
            'new_외국인(순매수/거래대금)','new_기타외국인(순매수/거래량)','new_기타외국인(순매수/거래대금)',
            'new_기타법인(순매수/거래량)','new_기타법인(순매수/거래대금)']]

In [71]:
df_re1.columns = ['해당월','기관투자자(매도/거래량)','기관투자자(매도/거래대금)',
              '개인(매도/거래량)','개인(매도/거래대금)','외국인(매도/거래량)',
            '외국인(매도/거래대금)','기타외국인(매도/거래량)','기타외국인(매도/거래대금)',
            '기타법인(매도/거래량)','기타법인(매도/거래대금)','기관투자자(매수/거래량)','기관투자자(매수/거래대금)',
             '개인(매수/거래량)','개인(매수/거래대금)','외국인(매수/거래량)',
            '외국인(매수/거래대금)','기타외국인(매수/거래량)','기타외국인(매수/거래대금)',
            '기타법인(매수/거래량)','기타법인(매수/거래대금)','기관투자자(순매수/거래량)','기관투자자(순매수/거래대금)',
             '개인(순매수/거래량)','개인(순매수/거래대금)','외국인(순매수/거래량)',
            '외국인(순매수/거래대금)','기타외국인(순매수/거래량)','기타외국인(순매수/거래대금)',
            '기타법인(순매수/거래량)','기타법인(순매수/거래대금)']

In [72]:
df_re1

,해당월,기관투자자(매도/거래량),기관투자자(매도/거래대금),개인(매도/거래량),개인(매도/거래대금),외국인(매도/거래량),외국인(매도/거래대금),기타외국인(매도/거래량),기타외국인(매도/거래대금),기타법인(매도/거래량),...,기관투자자(순매수/거래량),기관투자자(순매수/거래대금),개인(순매수/거래량),개인(순매수/거래대금),외국인(순매수/거래량),외국인(순매수/거래대금),기타외국인(순매수/거래량),기타외국인(순매수/거래대금),기타법인(순매수/거래량),기타법인(순매수/거래대금)
0,2018/09,463930,21738513,4124203,49722354,792249,30020323,7764,120386,45080,...,3279,635087,-7064,-33147,11161,-297485,58,-10004,-7434,-294452
1,2018/10,637169,28440410,5221373,55931757,1104143,40719057,10034,113849,49377,...,6896,1789725,-11124,1909966,-12437,-3998777,-112,10151,16777,288935
2,2018/11,631475,26752516,5180401,53664002,1116264,35096871,9109,107032,41709,...,-10267,-217884,-5996,-371686,5610,360165,-30,-6772,10683,236178
3,2018/12,567593,23321422,6065660,46923235,923756,28525485,11708,108432,63162,...,30972,1175187,-23569,-1233891,473,-50537,-122,-8567,-7754,117808
4,2019/01,767544,29659073,7564319,61777978,1021412,30812607,15415,148252,86508,...,-57579,-843145,-24751,-3229521,120564,4050033,-499,-16475,-37734,39108
5,2019/02,536465,21943755,5417014,45699594,857260,27118281,12318,118299,59498,...,-17872,541546,55291,-826496,-13077,140834,-693,-6077,-23649,150193
6,2019/03,635124,25774280,4810184,40946832,930013,29228049,9091,90111,73644,...,-26795,-994507,31582,590215,10674,300926,168,-397,-15628,103763
7,2019/04,697869,27004481,7916701,61205745,911913,29104189,17282,136022,145983,...,-20559,-654037,49547,-1747825,58509,2392135,-90,-10017,-87407,19744
8,2019/05,704402,25691163,9176499,50121982,1268337,36521584,18538,99900,75116,...,-45430,-462639,129047,2799941,-69187,-2480729,1192,13967,-15623,129460
9,2019/06,551935,19970200,9618449,41974243,999229,24613715,23850,139470,63224,...,15348,1511951,-15366,-1970077,22015,758298,-1454,-50071,-20543,-250101


In [73]:
df_re1.to_csv('.../월별_투자자별_매도매수순매수.csv',index=False,encoding='utf-8-sig')

################################################################################

In [74]:
df2 = maesoo_fin_fin
df2.head(5)

,해당월,기관투자자(매수/거래량),기관투자자(매수/거래대금),개인(매수/거래량),개인(매수/거래대금),외국인(매수/거래량),외국인(매수/거래대금),기타외국인(매수/거래량),기타외국인(매수/거래대금),기타법인(매수/거래량),...,기관투자자(순매수/거래량),기관투자자(순매수/거래대금),개인(순매수/거래량),개인(순매수/거래대금),외국인(순매수/거래량),외국인(순매수/거래대금),기타외국인(순매수/거래량),기타외국인(순매수/거래대금),기타법인(순매수/거래량),기타법인(순매수/거래대금)
0,2018/09,"467,208","22,373,600","4,117,139","49,689,208","803,411","29,722,838","7,822","110,382","37,646",...,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
1,2018/10,"644,065","30,230,135","5,210,249","57,841,723","1,091,707","36,720,280","9,922","124,000","66,153",...,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
2,2018/11,"621,208","26,534,632","5,174,406","53,292,315","1,121,874","35,457,036","9,080","100,260","52,392",...,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
3,2018/12,"598,565","24,496,609","6,042,091","45,689,344","924,230","28,474,948","11,585","99,864","55,408",...,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
4,2019/01,"709,965","28,815,928","7,539,567","58,548,458","1,141,976","34,862,640","14,916","131,777","48,773",...,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"


In [75]:
new_g11 = df2['기관투자자(매수/거래량)'].str.replace(',', '').astype('int64')
new_g12 = df2['기관투자자(매수/거래대금)'].str.replace(',', '').astype('int64')
new_g13 = df2['개인(매수/거래량)'].str.replace(',', '').astype('int64')
new_g14 = df2['개인(매수/거래대금)'].str.replace(',', '').astype('int64')
new_g15 = df2['외국인(매수/거래량)'].str.replace(',', '').astype('int64')
new_g16 = df2['외국인(매수/거래대금)'].str.replace(',', '').astype('int64')
new_g17 = df2['기타외국인(매수/거래량)'].str.replace(',', '').astype('int64')
new_g18 = df2['기타외국인(매수/거래대금)'].str.replace(',', '').astype('int64')
new_g19 = df2['기타법인(매수/거래량)'].str.replace(',', '').astype('int64')
new_g20 = df2['기타법인(매수/거래대금)'].str.replace(',', '').astype('int64')
new_g21 = df2['기관투자자(순매수/거래량)'].str.replace(',', '').astype('int64')
new_g22 = df2['기관투자자(순매수/거래대금)'].str.replace(',', '').astype('int64')
new_g23 = df2['개인(순매수/거래량)'].str.replace(',', '').astype('int64')
new_g24 = df2['개인(순매수/거래대금)'].str.replace(',', '').astype('int64')
new_g25 = df2['외국인(순매수/거래량)'].str.replace(',', '').astype('int64')
new_g26 = df2['외국인(순매수/거래대금)'].str.replace(',', '').astype('int64')
new_g27 = df2['기타외국인(순매수/거래량)'].str.replace(',', '').astype('int64')
new_g28 = df2['기타외국인(순매수/거래대금)'].str.replace(',', '').astype('int64')
new_g29 = df2['기타법인(순매수/거래량)'].str.replace(',', '').astype('int64')
new_g30 = df2['기타법인(순매수/거래대금)'].str.replace(',', '').astype('int64')

In [76]:
df2['new_기관투자자(매수/거래량)']=new_g11
df2['new_기관투자자(매수/거래대금)']=new_g12
df2['new_개인(매수/거래량)']=new_g13
df2['new_개인(매수/거래대금)']=new_g14
df2['new_외국인(매수/거래량)']=new_g15
df2['new_외국인(매수/거래대금)']=new_g16
df2['new_기타외국인(매수/거래량)']=new_g17
df2['new_기타외국인(매수/거래대금)']=new_g18
df2['new_기타법인(매수/거래량)']=new_g19
df2['new_기타법인(매수/거래대금)']=new_g20
df2['new_기관투자자(순매수/거래량)']=new_g21
df2['new_기관투자자(순매수/거래대금)']=new_g22
df2['new_개인(순매수/거래량)']=new_g23
df2['new_개인(순매수/거래대금)']=new_g24
df2['new_외국인(순매수/거래량)']=new_g25
df2['new_외국인(순매수/거래대금)']=new_g26
df2['new_기타외국인(순매수/거래량)']=new_g27
df2['new_기타외국인(순매수/거래대금)']=new_g28
df2['new_기타법인(순매수/거래량)']=new_g29
df2['new_기타법인(순매수/거래대금)']=new_g30

In [77]:
df_re2 = df2[['해당월','new_기관투자자(매수/거래량)','new_기관투자자(매수/거래대금)',
             'new_개인(매수/거래량)','new_개인(매수/거래대금)','new_외국인(매수/거래량)',
            'new_외국인(매수/거래대금)','new_기타외국인(매수/거래량)','new_기타외국인(매수/거래대금)',
            'new_기타법인(매수/거래량)','new_기타법인(매수/거래대금)','new_기관투자자(순매수/거래량)','new_기관투자자(순매수/거래대금)',
             'new_개인(순매수/거래량)','new_개인(순매수/거래대금)','new_외국인(순매수/거래량)',
            'new_외국인(순매수/거래대금)','new_기타외국인(순매수/거래량)','new_기타외국인(순매수/거래대금)',
            'new_기타법인(순매수/거래량)','new_기타법인(순매수/거래대금)']]

In [78]:
df_re2.columns = ['해당월','기관투자자(매수/거래량)','기관투자자(매수/거래대금)',
             '개인(매수/거래량)','개인(매수/거래대금)','외국인(매수/거래량)',
            '외국인(매수/거래대금)','기타외국인(매수/거래량)','기타외국인(매수/거래대금)',
            '기타법인(매수/거래량)','기타법인(매수/거래대금)','기관투자자(순매수/거래량)','기관투자자(순매수/거래대금)',
             '개인(순매수/거래량)','개인(순매수/거래대금)','외국인(순매수/거래량)',
            '외국인(순매수/거래대금)','기타외국인(순매수/거래량)','기타외국인(순매수/거래대금)',
            '기타법인(순매수/거래량)','기타법인(순매수/거래대금)']

In [79]:
df_re2

,해당월,기관투자자(매수/거래량),기관투자자(매수/거래대금),개인(매수/거래량),개인(매수/거래대금),외국인(매수/거래량),외국인(매수/거래대금),기타외국인(매수/거래량),기타외국인(매수/거래대금),기타법인(매수/거래량),...,기관투자자(순매수/거래량),기관투자자(순매수/거래대금),개인(순매수/거래량),개인(순매수/거래대금),외국인(순매수/거래량),외국인(순매수/거래대금),기타외국인(순매수/거래량),기타외국인(순매수/거래대금),기타법인(순매수/거래량),기타법인(순매수/거래대금)
0,2018/09,467208,22373600,4117139,49689208,803411,29722838,7822,110382,37646,...,3279,635087,-7064,-33147,11161,-297485,58,-10004,-7434,-294452
1,2018/10,644065,30230135,5210249,57841723,1091707,36720280,9922,124000,66153,...,6896,1789725,-11124,1909966,-12437,-3998777,-112,10151,16777,288935
2,2018/11,621208,26534632,5174406,53292315,1121874,35457036,9080,100260,52392,...,-10267,-217884,-5996,-371686,5610,360165,-30,-6772,10683,236178
3,2018/12,598565,24496609,6042091,45689344,924230,28474948,11585,99864,55408,...,30972,1175187,-23569,-1233891,473,-50537,-122,-8567,-7754,117808
4,2019/01,709965,28815928,7539567,58548458,1141976,34862640,14916,131777,48773,...,-57579,-843145,-24751,-3229521,120564,4050033,-499,-16475,-37734,39108
5,2019/02,518593,22485301,5472305,44873098,844183,27259116,11624,112222,35849,...,-17872,541546,55291,-826496,-13077,140834,-693,-6077,-23649,150193
6,2019/03,608329,24779774,4841766,41537047,940687,29528974,9258,89713,58016,...,-26795,-994507,31582,590215,10674,300926,168,-397,-15628,103763
7,2019/04,677309,26350444,7966248,59457920,970422,31496323,17192,126005,58577,...,-20559,-654037,49547,-1747825,58509,2392135,-90,-10017,-87407,19744
8,2019/05,658973,25228524,9305547,52921922,1199150,34040855,19730,113867,59493,...,-45430,-462639,129047,2799941,-69187,-2480729,1192,13967,-15623,129460
9,2019/06,567284,21482151,9603083,40004166,1021244,25372014,22396,89398,42681,...,15348,1511951,-15366,-1970077,22015,758298,-1454,-50071,-20543,-250101


In [80]:
df_re2.to_csv('C:/Users/starfish/Desktop/월별_투자자별_매수순매수.csv',index=False,encoding='utf-8-sig')

--------------------------------------------------------------

In [43]:
soonmaesoo = pd.read_csv("soonmaesoo.csv",encoding='CP949')
soonmaesoo

,통계표,항목명1,항목명2,단위,변환,2018/09,2018/10,2018/11,2018/12,2019/01,...,2019/11,2019/12,2020/01,2020/02,2020/03,2020/04,2020/05,2020/06,2020/07,2020/08
0,6.1.3 투자자별 주식거래,기관투자자(순매수),거래량,천주,원자료,"3,279","6,896","-10,267","30,972","-57,579",...,"-1,691","44,058","-139,180","-98,752","-97,410","-8,194","6,393","-95,352","-89,997","-134,485"
1,6.1.3 투자자별 주식거래,기관투자자(순매수),거래대금,백만원,원자료,"635,087","1,789,725","-217,884","1,175,187","-843,145",...,"2,309,691","2,762,709","-5,074,181","-2,236,091","116,646","-13,690","-188,143","-2,710,753","-3,063,739","-3,563,968"
2,6.1.3 투자자별 주식거래,개인(순매수),거래량,천주,원자료,"-7,064","-11,124","-5,996","-23,569","-24,751",...,"119,553","-50,692","243,962","250,006","256,121","162,863","165,750","210,715","128,070","181,793"
3,6.1.3 투자자별 주식거래,개인(순매수),거래대금,백만원,원자료,"-33,147","1,909,966","-371,686","-1,233,891","-3,229,521",...,"138,619","-3,827,536","4,483,045","4,897,356","11,186,930","3,812,420","3,783,481","3,814,351","2,238,918","6,170,756"
4,6.1.3 투자자별 주식거래,외국인(순매수),거래량,천주,원자료,"11,161","-12,437","5,610",473,"120,564",...,"-105,346","9,432","-77,212","-132,467","-254,574","-143,932","-166,660","-111,426","45,663","-34,690"
5,6.1.3 투자자별 주식거래,외국인(순매수),거래대금,백만원,원자료,"-297,485","-3,998,777","360,165","-50,537","4,050,033",...,"-3,170,796","618,387","304,720","-3,313,159","-12,555,010","-4,100,079","-3,883,794","-1,218,825","1,079,096","-2,846,887"
6,6.1.3 투자자별 주식거래,기타외국인(순매수),거래량,천주,원자료,58,-112,-30,-122,-499,...,660,-385,-338,105,"2,257","-1,349","2,517","1,582","-1,578",-120
7,6.1.3 투자자별 주식거래,기타외국인(순매수),거래대금,백만원,원자료,"-10,004","10,151","-6,772","-8,567","-16,475",...,"-4,391","-18,292","15,476","4,763","37,581","4,127",656,"19,588","-34,432","14,770"
8,6.1.3 투자자별 주식거래,기타법인(순매수),거래량,천주,원자료,"-7,434","16,777","10,683","-7,754","-37,734",...,"-13,176","-2,414","-27,232","-18,892","93,607","-9,389","-8,000","-5,519","-82,158","-12,499"
9,6.1.3 투자자별 주식거래,기타법인(순매수),거래대금,백만원,원자료,"-294,452","288,935","236,178","117,808","39,108",...,"726,878","464,732","270,941","647,130","1,213,853","297,222","287,799","95,639","-219,842","225,329"


In [10]:
soonmaesoo.T

,0,1,2,3,4,5,6,7,8,9
통계표,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래
항목명1,기관투자자(순매수),기관투자자(순매수),개인(순매수),개인(순매수),외국인(순매수),외국인(순매수),기타외국인(순매수),기타외국인(순매수),기타법인(순매수),기타법인(순매수)
항목명2,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금
단위,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원
변환,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료
2018/09,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
2018/10,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
2018/11,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
2018/12,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
2019/01,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"


In [44]:
new_soonmaesoo = soonmaesoo.T
new_soonmaesoo.head(7)

,0,1,2,3,4,5,6,7,8,9
통계표,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래
항목명1,기관투자자(순매수),기관투자자(순매수),개인(순매수),개인(순매수),외국인(순매수),외국인(순매수),기타외국인(순매수),기타외국인(순매수),기타법인(순매수),기타법인(순매수)
항목명2,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금
단위,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원
변환,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료
2018/09,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
2018/10,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"


In [45]:
re_soonmaesoo = new_soonmaesoo.reset_index()
re_soonmaesoo.head(7)

,index,0,1,2,3,4,5,6,7,8,9
0,통계표,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래,6.1.3 투자자별 주식거래
1,항목명1,기관투자자(순매수),기관투자자(순매수),개인(순매수),개인(순매수),외국인(순매수),외국인(순매수),기타외국인(순매수),기타외국인(순매수),기타법인(순매수),기타법인(순매수)
2,항목명2,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금,거래량,거래대금
3,단위,천주,백만원,천주,백만원,천주,백만원,천주,백만원,천주,백만원
4,변환,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료,원자료
5,2018/09,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
6,2018/10,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"


In [46]:
soonmaesoo_ = re_soonmaesoo.drop([re_soonmaesoo.index[0],re_soonmaesoo.index[1],re_soonmaesoo.index[2],re_soonmaesoo.index[3],re_soonmaesoo.index[4]])
soonmaesoo_.head(7)

,index,0,1,2,3,4,5,6,7,8,9
5,2018/09,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
6,2018/10,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
7,2018/11,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
8,2018/12,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
9,2019/01,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"
10,2019/02,"-17,872","541,546","55,291","-826,496","-13,077","140,834",-693,"-6,077","-23,649","150,193"
11,2019/03,"-26,795","-994,507","31,582","590,215","10,674","300,926",168,-397,"-15,628","103,763"


In [47]:
soonmaesoo_.columns = ['해당월',
                  '기관투자자(순매수/거래량)','기관투자자(순매수/거래대금)',
                  '개인(순매수/거래량)','개인(순매수/거래대금)',
                  '외국인(순매수/거래량)','외국인(순매수/거래대금)',
                  '기타외국인(순매수/거래량)','기타외국인(순매수/거래대금)',
                  '기타법인(순매수/거래량)','기타법인(순매수/거래대금)']

In [48]:
soonmaesoo_.head(5)

,해당월,기관투자자(순매수/거래량),기관투자자(순매수/거래대금),개인(순매수/거래량),개인(순매수/거래대금),외국인(순매수/거래량),외국인(순매수/거래대금),기타외국인(순매수/거래량),기타외국인(순매수/거래대금),기타법인(순매수/거래량),기타법인(순매수/거래대금)
5,2018/09,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
6,2018/10,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
7,2018/11,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
8,2018/12,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
9,2019/01,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"


In [49]:
soonmaesoo_fin = soonmaesoo_
soonmaesoo_fin

,해당월,기관투자자(순매수/거래량),기관투자자(순매수/거래대금),개인(순매수/거래량),개인(순매수/거래대금),외국인(순매수/거래량),외국인(순매수/거래대금),기타외국인(순매수/거래량),기타외국인(순매수/거래대금),기타법인(순매수/거래량),기타법인(순매수/거래대금)
5,2018/09,"3,279","635,087","-7,064","-33,147","11,161","-297,485",58,"-10,004","-7,434","-294,452"
6,2018/10,"6,896","1,789,725","-11,124","1,909,966","-12,437","-3,998,777",-112,"10,151","16,777","288,935"
7,2018/11,"-10,267","-217,884","-5,996","-371,686","5,610","360,165",-30,"-6,772","10,683","236,178"
8,2018/12,"30,972","1,175,187","-23,569","-1,233,891",473,"-50,537",-122,"-8,567","-7,754","117,808"
9,2019/01,"-57,579","-843,145","-24,751","-3,229,521","120,564","4,050,033",-499,"-16,475","-37,734","39,108"
10,2019/02,"-17,872","541,546","55,291","-826,496","-13,077","140,834",-693,"-6,077","-23,649","150,193"
11,2019/03,"-26,795","-994,507","31,582","590,215","10,674","300,926",168,-397,"-15,628","103,763"
12,2019/04,"-20,559","-654,037","49,547","-1,747,825","58,509","2,392,135",-90,"-10,017","-87,407","19,744"
13,2019/05,"-45,430","-462,639","129,047","2,799,941","-69,187","-2,480,729","1,192","13,967","-15,623","129,460"
14,2019/06,"15,348","1,511,951","-15,366","-1,970,077","22,015","758,298","-1,454","-50,071","-20,543","-250,101"


In [62]:
soonmaesoo_fin_fin = soonmaesoo_fin.reset_index()
soonmaesoo_fin_fin.drop(['index'],axis='columns',inplace=True)

In [63]:
soonmaesoo_fin_fin.to_csv('.../월별_투자자별_순매수.csv',index=False,encoding='utf-8-sig')

In [ ]:
# 순매수 데이터타입 변경은 이미 진행...